In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import asyncio
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, List

# Define state
class State(TypedDict):
    question: str
    answer: List[str]

# Our LLM (no streaming)
llm = ChatOpenAI(model="gpt-4o-mini")

# Node 1: Generate full answer
async def generate(state: State) -> State:
    print(f"\n[Generate] Question: {state['question']}")
    response = await llm.ainvoke(state["question"])   # no streaming, full response
    return {"answer": [response.content]}

# Node 2: Summarize answer
async def summarize(state: State) -> State:
    print("\n[Summarize] Processing...")
    summary = await llm.ainvoke("Summarize in one line: " + "".join(state["answer"]))
    return {"answer": [summary.content]}

# Build Graph
workflow = StateGraph(State)
workflow.add_node("generate", generate)
workflow.add_node("summarize", summarize)

workflow.add_edge(START, "generate")
workflow.add_edge("generate", "summarize")
workflow.add_edge("summarize", END)

# Compile graph
app = workflow.compile()

# One request handler
async def handle_request(question: str):
    inputs = {"question": question}
    print(f"\n===== Handling request: {question} =====")
    
    # No streaming — just run the graph
    result = await app.ainvoke(inputs)
    print(f"\nFinal Output for '{question}': {result['answer']}")
    return result

# Run multiple requests concurrently
async def main():
    questions = [
        "Explain async and streaming in simple terms",
        "What is LangGraph and why is it useful?",
        "Summarize the difference between sync and async programming",
    ]
    
    # Run all requests at once
    results = await asyncio.gather(*(handle_request(q) for q in questions))
    print("\n=== ALL RESULTS ===")
    for r in results:
        print(r)

# In Jupyter/Notebook: just use
await main()

# In a normal script, replace with:
# if __name__ == "__main__":
#     asyncio.run(main())



===== Handling request: Explain async and streaming in simple terms =====

===== Handling request: What is LangGraph and why is it useful? =====

===== Handling request: Summarize the difference between sync and async programming =====

[Generate] Question: Explain async and streaming in simple terms

[Generate] Question: What is LangGraph and why is it useful?

[Generate] Question: Summarize the difference between sync and async programming

[Summarize] Processing...

Final Output for 'Summarize the difference between sync and async programming': ['Synchronous programming executes tasks in a linear, blocking manner, while asynchronous programming enables concurrent task execution without blocking, enhancing efficiency and responsiveness but increasing complexity.']

[Summarize] Processing...

[Summarize] Processing...

Final Output for 'What is LangGraph and why is it useful?': ['LangGraph is a framework that enhances the interaction and efficacy of language models in NLP tasks by ut

In [ ]:
import asyncio
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, List

# Define state
class State(TypedDict):
    question: str
    answer: List[str]

# Our LLM
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

# Node 1: Async Agent (generates answer tokens)
async def generate(state: State) -> State:
    print(f"\n[Generate] Question: {state['question']}")
    tokens = []
    async for chunk in llm.astream(state["question"]):
        if chunk.content:   # content is str
            print(chunk.content, end="", flush=True)
            tokens.append(chunk.content)
    print("\n--- generation complete ---")
    return {"answer": tokens}

# Node 2: Async Post-processor
async def summarize(state: State) -> State:
    print("\n[Summarize] Processing...")
    summary = await llm.ainvoke("Summarize in one line: " + "".join(state["answer"]))
    return {"answer": [summary.content]}

# Build Graph
workflow = StateGraph(State)
workflow.add_node("generate", generate)
workflow.add_node("summarize", summarize)

workflow.add_edge(START, "generate")
workflow.add_edge("generate", "summarize")
workflow.add_edge("summarize", END)

# Compile graph
app = workflow.compile()

# One request handler
async def handle_request(question: str):
    inputs = {"question": question}
    
    print(f"\n===== Handling request: {question} =====")
    async for event in app.astream(inputs):
        print(f"\n[STREAM EVENT] {event}")

    # Final state
    result = await app.ainvoke(inputs)
    print(f"\nFinal Output for '{question}': {result['answer']}")
    return result

# Run multiple requests concurrently
async def main():
    questions = [
        "Explain async and streaming in simple terms",
        "What is LangGraph and why is it useful?",
        "Summarize the difference between sync and async programming",
    ]
    
    # Run all requests at once
    results = await asyncio.gather(*(handle_request(q) for q in questions))
    print("\n=== ALL RESULTS ===")
    for r in results:
        print(r)

# Entry point
if __name__ == "__main__":
    await main()



===== Handling request: Explain async and streaming in simple terms =====

===== Handling request: What is LangGraph and why is it useful? =====

===== Handling request: Summarize the difference between sync and async programming =====

[Generate] Question: Explain async and streaming in simple terms

[Generate] Question: What is LangGraph and why is it useful?

[Generate] Question: Summarize the difference between sync and async programming
The primary difference between synchronous (LangGraph issync)Sure! Let's break and asynchronous (async down "async") programming and " lies in a framework designed tostreaming facilitate the how tasks development of applications that leverage advanced" in simple terms language models:

### are executed and handled, particularly and graph Async (Asynchronous in relation to blocking)

** and nonAsync**,-blocking-based structures short for operations.

### S asynchronous, is aynchronous Programming way of:
- handling tasks or operations **Execution w